In [5]:
import numpy as np
import torch

from model.resnet_model import CNNClassifier
from torch.utils.data import DataLoader
from pathlib import Path
from ml_utils import CustomDataset

from sklearn.metrics import f1_score, accuracy_score
from torch.nn import CrossEntropyLoss

from scipy.stats import entropy

In [6]:
device = "mps"

In [7]:
test_dt = CustomDataset(data_path=Path("../datasets/car_dataset"), portion="test")
test_loader = DataLoader(test_dt, batch_size=64, shuffle=True, num_workers=4)

cls = CNNClassifier(num_classes=196, pretrained=True, model_name="resnet18", freeze_layers=0)
cls.load_state_dict(torch.load("../experiments/results/r18_pretrained_fre0.pth", weights_only=True))
cls.eval()
cls = cls.to(device)

In [8]:
from tqdm import tqdm

preds, trues, entropy_list = [], [], []
criterion = CrossEntropyLoss().to(device)

for idx, (x, y) in enumerate(tqdm(test_loader), 1):
    x, y = x.to(device), y.to(device)
    x.requires_grad_(True)

    loss = criterion(cls(x), y)
    cls.zero_grad()
    loss.backward()

    with torch.no_grad():
        adv_logits = cls(x + .007 * torch.sign(x.grad))
        entropy_list.append(entropy(torch.softmax(adv_logits, dim=1).cpu().numpy()))

    predictions = torch.argmax(adv_logits, dim=1)
    preds.append(predictions)
    trues.append(y)

# Concatenate all predictions, true labels, and losses
preds = torch.cat(preds).cpu().numpy()
trues = torch.cat(trues).cpu().numpy()
entropy_list = torch.tensor(np.asarray(entropy_list))

f1_mean = f1_score(trues, preds, average='weighted')
acc_mean = accuracy_score(trues, preds)

100%|██████████| 126/126 [04:11<00:00,  1.99s/it]
/var/folders/09/gz7wkjb96v1dzq_xpbf9gv5h0000gn/T/ipykernel_13975/2174783170.py:25: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_new.cpp:257.)
  entropy_list = torch.tensor(entropy_list)


In [9]:
print(entropy_list.mean(), entropy_list.std())

tensor(2.6242) tensor(0.8007)


In [10]:
f1_mean

0.12186538816469347

In [11]:
acc_mean

0.11677652033329187